In [1]:
import numpy as np
import pandas as pd
import random
import time
from sklearn.model_selection import RepeatedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import linalg
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
def ARDPG(n,k):
    A = np.zeros((n,n))
    X = np.random.uniform(0.0, 1.0, (n,1))
    for i in range(0,n):
        if y[i] == 1:
            X[i,:] = np.random.beta(1,3)
        elif y[i] == 2:
            X[i,:] = np.random.beta(3,3)
        else:
            X[i,:] = np.random.beta(3,1)
    P=np.matmul(X,X.transpose())
    for i in range(0,n):
        for j in range(1,n):
            A[i,j]=np.random.binomial(1,P[i,j])
    for i in range(0,n):
        for j in range(0,i):
            A[i,j] = A[j,i]
            A[i,i]=0
    return A

In [3]:
def gety(n):
    y = np.zeros((n,1))
    #np.random.seed(2)
    for i in range(0,n):
        y[i,0] = np.random.choice(np.arange(1, 4), p=[0.2, 0.3,0.5])
    return(y)

In [4]:
def EncoderLDA(A):
#Split Data
    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)
    for train_index, test_index in rkf.split(A):
         #print("TRAIN:", train_index, "TEST:", test_index)
         X_train, X_test = A[train_index], A[test_index]
         y_train, y_test = y[train_index], y[test_index]
         X1 = X_train.transpose()
         X_train, X_test = X1[train_index], X1[test_index]
#ASE*LDA
    begin1 = time.time()
    U, s, VT = np.linalg.svd(A)
    newx=([])
    d=3
    newx=np.concatenate(U[:,0:d]*s[0:d]**0.5)
    newx=np.reshape(newx,(n,d))
    clf2 = LinearDiscriminantAnalysis()
    #print(np.shape(newx))
    newx_train, newx_test =newx[train_index], newx[test_index]
    clf2.fit(newx_train, y_train.ravel())
    ASELDAACC=clf2.score(newx_test, y_test.ravel())
    #time.sleep(2) 
    end1 = time.time()
    ASELDATIME=end1-begin1
    eigenvalues=s**2
    Top=eigenvalues[:10]
#nk,w,Z
    begin5 = time.time()
    global k
    nk = np.zeros((1,k))
    for i in range(0,len(y_train)-1):
        nk[0,int(y_train[i,0]-1)]=nk[0,int(y_train[i,0]-1)]+1
    w = np.zeros((int(np.size(y_train)),k))
    for i in range(0,int(np.size(y_train))):
        k=int(y_train[i])
        w[i][k-1]=1/nk[0,k-1]
    z_train=np.matmul(X_train,w)
    z_test = np.matmul(X_test,w)
    #time.sleep(5)
    end5 = time.time()
    zTIME=end5 - begin5
#AEE*KNN
    begin2 = time.time()
    neigh = KNeighborsClassifier(weights='uniform',metric='euclidean')
    neigh.fit(z_train, y_train.ravel())
    AEEKNNACC = neigh.score(z_test, y_test)
    #time.sleep(3) 
    end2 = time.time()
    AEEKNNTIME=end2-begin2
#AEE*LDA
    begin3 = time.time()
    clf4 = LinearDiscriminantAnalysis()
    clf4.fit(z_train,y_train.ravel())
    AEELDAACC = clf4.score(z_test, y_test.ravel())
    #time.sleep(4) 
    end3 = time.time()
    AEELDATIME=end3-begin3
#Get result
    Acc=np.array([ASELDAACC,AEEKNNACC,AEELDAACC])
    Time=np.array([ASELDATIME,AEEKNNTIME+zTIME,AEELDATIME+zTIME])
    print(f"When n = {n}")
    print(f"Accuracy for each method: {Acc}")
    print(f"Time for each method: {Time}")

In [5]:
for n in range (100,3100,100):
    k=3
    y=gety(n)
    A=ARDPG(n,k=k)
    EncoderLDA(A)

When n = 100
Accuracy for each method: [0.7  0.65 0.65]
Time for each method: [0.01365876 0.00294209 0.00154233]
When n = 200
Accuracy for each method: [0.65  0.625 0.675]
Time for each method: [0.01041412 0.00429797 0.00208092]
When n = 300
Accuracy for each method: [0.66666667 0.55       0.61666667]
Time for each method: [0.02868891 0.00512695 0.00243092]
When n = 400
Accuracy for each method: [0.675  0.675  0.7125]
Time for each method: [0.04200602 0.00585699 0.00449562]
When n = 500
Accuracy for each method: [0.69 0.67 0.7 ]
Time for each method: [0.07512307 0.00696898 0.00326824]
When n = 600
Accuracy for each method: [0.625 0.625 0.65 ]
Time for each method: [0.11924124 0.00811172 0.00422955]
When n = 700
Accuracy for each method: [0.61428571 0.59285714 0.60714286]
Time for each method: [0.17136717 0.01062298 0.00416708]
When n = 800
Accuracy for each method: [0.65625 0.575   0.675  ]
Time for each method: [0.26198983 0.01027799 0.00468731]
When n = 900
Accuracy for each method: 